#### Copyright IBM All Rights Reserved.

# Using the IBM Db2 on Cloud REST API

This notebook includes basic Python examples for the REST API to Db2 on Cloud for exploring data.

The official documentation of the DB2 RestAPI ist here:
https://cloud.ibm.com/apidocs/db2-on-cloud




## Prerequisites:
1. Python 3.6 and above
2. Db2 on Cloud instance (using free-tier option)
3. Data already loaded in your Db2 instance
4. Db2 connection credentials on hand

## Required libraries

To run the examples in this notebook, you need to install the following libraries:

- `pandas`: For data manipulation and analysis.
- `requests`: For making HTTP calls.

We also use the `pandas` library. This library should already be installed if you are able to use this notebook.

## 1. Setting up

We are going to import the required libraries and create a variable that stores the base URL for the REST API. Update this variable to a value that matches your environment.

In [ ]:
import pandas as pd
import requests
from pandas.io.json import json_normalize

### Load Db2 Connection Information
The first step is to load the Db2 connection information into this notebook. Copy your service credentials provided in JSON format into the curly brackets. The definition of db2id should look like this:

```json
db2id = {
  "db": "myDatabaseName",
  "host": "anydbservice.mycloud.net",
  "hostname": "anydbservice.mycloud.net",
  "https_url": "https://anydbservice.mycloud.net",
  "port": 50000,
  "userid" : "xyz12345",
  "password" :"secretPassword"
}
```

In [ ]:
db2id = {
}


### Calling Restful Services

There are two types of RESTful calls that are used with Db2 on Cloud:

* **`GET`** - Get results from a SQL request
* **`POST`** - Request an Access token, or issue an SQL command

All RESTful calls require the host IP address and the service URL:
* **`host`** - This is the IP address of the machine that is hosting Db2 on Cloud
* **`api`** - The API library that is being used to communicate with Db2 on Cloud
* **`service`** - The service (API) that is being requested. 

As mentioned previously, the API library that is used for Db2 on Cloud is `/dbapi/v3`. The full URL has the following format:

```
https:// + host + api + service
```

If your host was xyz.abc.com and we are using the `/auth/tokens` service, the full URL would be:

```
https://xyz.abc.com/dbapi/v3/auth/tokens
```

This URL is then placed into a RESTful call that has one of the following formats:

```
RESTful.call( host + api + service, json=data)
RESTful.call( host + api + service, headers=tokens, json=data)
```

The first form of the RESTful call sends parameters via the **`json=data`** keyword. This format of RESTful call is used to generate an access token. The second form of the RESTful call will supply parameters through the **`json=data`** keyword, and also supply the access token via the **`headers=tokens`** keyword. Any SQL request needs this access token to be supplied, otherwise the Db2 on Cloud service will not be able to authenticate the RESTful call.

### Generating an Access Token
The first step in generating an access token is to build the URL for the service. From a programming perspective, the host URL will be combined with the API library to form the stub for all future RESTful URL requests. The HOST IP address is derived from the Db2 connect parameters.

In [ ]:
api = '/dbapi/v3'
host = db2id['https_url'] + api
print( "Host URL = " + host)

Next we need to build the JSON parameter structure required for the authentication call. This JSON parameter needs to contain the following information:
```json
{
    "userid" : "xyz12345",
    "password" :"randompassword"
}
```
The next statement will create a Python variable that contains your userid and password from in the Db2 connection.

In [ ]:
userinfo = {
    "userid" : db2id['username'],
    "password" : db2id['password']
}

The final step before we issue the RESTful call is to load the Python RESTful library into the notebook and define what service we are going to use in our RESTful call.

In [ ]:
import requests
service = "/auth/tokens"

The final format of the RESTful call to request an access token looks like this:

RESTful.post(host + service,json=userinfo)
If this call returns successfully, the post request will return a structure that looks like this:

{
  "userid": "STRING",
  "token": "STRING"
}
Normally the requests call would be assigned to a variable, i.e x = requests.post(). A quick test to see if the RESTful call was successful is to compare x.ok to True. It is also good coding practice to surround the RESTful calls in a try/except block to catch any major errors that may occur.

Execute the next statement to retrieve an access token.

In [ ]:
r = requests.post(host + service, json=userinfo)

We check to see if the results of the call are okay. OK means that an HTTP return code of 200 was returned from the service which means that a valid call was made and that it was successful. If the HTTP return code is anything else, you would get a value of False.

In [ ]:
r.ok

The other option is to view the HTTP return code. Some of the possible values are:
* 400 - Bad request which means the parameters sent were incorrect or missing
* 401 - Unauthorized request
* 403 - Forbidden - Probably the wrong service or URL
* 404 - Not found - Probably the wrong service or URL

The `401` code is returned from the `/auth/tokens` service when an incorrect userid or password are sent to it. The previous call succeeded, so we can check the return code using `r.status_code`. 

In [ ]:
r.status_code

Since the status code is 200, we can examine the results returned by the RESTful call by checking the json structure in the answer set since this will contain the access token.

In [ ]:
r.json()

Now we can assign the access token to a variable that we will use for all of the remaining calls.

In [ ]:
access_token = r.json()['token']

The access token is valid for about an hour which means that you will need to refresh it if you are starting off in the middle of a notebook after having been away for a while. The next statement will create some invalid credentials and display the RESTful code and the JSON error structure that is returned.

### Request Access Token Routine
To simplify the process of getting an access token, the following Python routine will take the `db2id` structure and return an access token. In the event of an error, the routine will print the error message and return `None`. Note that the API library is imbedded in the routine so that if the API changes, the code must be updated.

In [ ]:
def get_token(db2id):
    
    import requests
    
    # Set up structures required for the RESTful call
    api = '/dbapi/v4'
    service = "/auth/tokens"
    host = db2id['https_url'] + api
    userinfo = {
        "userid" : db2id['username'],
        "password" : db2id['password']
    }
    
    # Call the RESTful service
    try:
        r = requests.post(host + service, json=userinfo)
    except Exception as err:
        print("RESTful call failed. Detailed information follows.")
        print(err)
        return None
    
    # Check for Invalid credentials
    if (r.status_code == 401): # There was an error with the authentication
        print("RESTful called failed.")
        message = r.json()['message']
        print(message)
        return None
    
    # Check for anything other than 200/401
    if (r.status_code != 200): # Some other failure
        print("RESTful called failed. Detailed information follows.")
        print(r.json())
        return None
    
    # Retrieve the access token
    try:
        access_token = r.json()['token']
    except:
        print("RESTful call did not return an access token.")
        print(r.json())
        return None
    
    return access_token

From a convenience perspective, we can add the access token to the Db2 credentials that we have. This way we only need to pass the Db2id to other routines rather than having the access token and the Db2id.

In [ ]:
db2id['access_token'] = get_token(db2id)

### Executing SQL Statements
There are two steps required to access data from Db2 on Cloud. The first RESTful call will post the SQL statements to Db2 on Cloud to be executed. It is possible to run multiple statements at once, but for the purposes of this lab we will only submit one statement at a time. 

The structure of the SQL request is:
```json
{
    "commands" : "sql",
    "limit" : x,
    "separator" : ";",
    "stop_on_error" : "yes"
}

```
The parameters are set as follows:

* commands - The SQL script to be executed (could be multiple statements).
* limit - Maximum number of rows that will be fetched for each result set.
* separator - SQL statement terminator. A character that is used to mark the end of a SQL statement when the provided SQL script contains multiple statements.
* stop_on_error - If 'yes', the job stops executing at the first statement that returns an error. If 'no', the job continues executing if one or more statements returns an error.

The service that is called to execute SQL statements is `/sql_jobs`. The RESTful post request has following format:
```
request.post(host + service, headers=authtoken, json=sql)
```
The `headers=authtoken` contains the authentication token that was retrieved in the first step and `json=sql` refers to the SQL statement and parameters required to execute it.

The value that is returned from the service is:
```json
{
  "id": "STRING",
  "commands_count": "INTEGER",
  "limit": "INTEGER"
}
```
The important piece of information is the `id` field which we need to track the execution of the SQL command(s). The first step that we need to take is to create the authentication header. The header has the format:
```json
{'Authorization': "Bearer " + token, 'X-DB-Profile': profile_name}
```
The format may look strange but this is the standard for passing tokens between a client and server. The `profile_name` is defined as part of the Db2 Console, which in this case just translates to the database name.

The next statement will create the authentication header.

In [ ]:
auth_header = {
    "Authorization" : "Bearer " + db2id['access_token'], 'X-DB-Profile': db2id['db']
}

Now we can create an SQL command with the associated parameters.

In [ ]:
sql_command = {
    "commands" : "SELECT tabname, card, npages, fpages FROM syscat.tables WHERE tabschema = upper('" + db2id['username']  + "')",
    "limit" : 1000,
    "separator" : ";",
    "stop_on_error" : "yes"
}

At this point we have enough information to request that the SQL be executed for us. We need to supply the authentication token and the SQL request.

In [ ]:
service = "/sql_jobs"
r = requests.post(host + service, headers=auth_header, json=sql_command)

In [ ]:
r.json()

In [ ]:
sql_command

We will check the return code (201) and if it is okay we will display the json results that are returned.

In [ ]:
if r.status_code == 201:
    print(r.json())
else:
    print("Something went wrong with the call!")

The `id` field is what we need to keep track of. The `id` is used to retrieve the answer set(s) that are associated with the SQL call. The value will be saved into a Python variable for future use.

In [ ]:
jobid = r.json()['id']

### Retrieving an Answer Set
Now that the SQL statement has been sent off for execution, we must request the results. The RESTful API is exactly the same as requesting the SQL to be run, but you need to add the job id to the end of the service. Instead of using a `POST` request we use `GET`. The format of the command is:
```
request.get(host + service + "/" + jobid, headers=authtoken)
```
When the call is successful, the result contains the following information:
```json
{
  "id": "STRING",
  "status": "STRING",
  "results": {
    "command": "STRING",
    "columns": "ARRAY",
    "rows": "ARRAY",
    "rows_count": "INTEGER",
    "limit": "INTEGER",
    "last_inserted": "INTEGER",
    "rows_affected": "INTEGER",
    "runtime_seconds": "DOUBLE",
    "error": "STRING"
  }
}
```
The `status` field contains either `completed`,`running`, or `failed`. It is possible that you only get an intermediate result set (perhaps because the answer set is still being gathered) so `running` will be returned. When the status is `running` you may already have some data in the `results` field. You must retrieve this data before issuing another request. The data that is returned is not cumulative which means any results that are returned in the RESTful call are lost on the next call. A general strategy would be to write code similar to:
```
r = restful.get(...)
if (r.status == "failed") leave...
rows = get rows from r
while r.status == "running"
    r = restful.get(...)
    rows = get rows from r
```
Since we only requested a count of rows, the following code should be successful (200).

In [ ]:
r = requests.get(host + service + "/" + jobid,headers=auth_header)
if r.status_code == 200:
    print(r.json())
else:
    print("Something went wrong with the call!")

The column names and rows of the result set are found in the 'results' field and the rows are represented as an array of values. To extract the column names use the following syntax.

In [ ]:
results = r.json()['results']
columns = results[0]['columns']
rows = results[0]['rows']
print('Column Names')
print(columns)
print('Rows')
print(rows)

The column names are in a single-dimensional array, while the rows are a 2-dimensional array. If you are using pandas dataframes to manipulate data, you can convert the array values using the following syntax. The last statement converts the character columns into numeric values. The RESTful call returns the answer set as strings so this step makes it easier to plot data.

In [ ]:
df = pd.DataFrame(data=rows,columns=columns)
cols = df.columns
df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')

Now that we have the data in dataframe format, just referring to the variable will print it in a nicer format.

In [ ]:
df

### Simplifying Data Retrieval
To make the process of getting an answer back easier, the following code will take all of the previous steps and encapsulate them into a single call. The function has the following format:
```
restSQL(db2id,sql)
```
The function will return a pandas dataframe as a result set. The parameters are:
* db2id - The id information from the Db2 Cloud service which includes the access_token created earlier
* sql - The SQL to be executed (single statement only)

One of the issues when using access tokens is that they expire after a certain period of time. This makes the access more secure since a token will expire in the event that it was accidentially released externally. However, this means that your queries may not work after a period of time because of an expired access token. If you do get an error with the SQL call, you will need to request a new access token with the following call:
```python
db2id['access_token'] = get_token(db2id)
```
The code in this function also checks to make sure that the SQL has completed execution. There is a possibility that the SQL will run for a long period of time so the function needs to continuously poll the server with RESTful calls until the SQL has completed.

In [ ]:
def restSQL(db2id,sql):
    
    import requests
    import pandas
    
    api = '/dbapi/v3'
    host = db2id["https_url"] + api
    
    authid = {"Authorization" : "Bearer " + db2id['access_token'], 'X-DB-Profile': db2id['db']}
    
    # Step 1: Build the SQL query and get the Job ID
    
    service = "/sql_jobs"

    request = {
        "commands" : sql,
        "limit" : 10000,
        "separator" : ";",
        "stop_on_error" : "yes"
    }
    
    try:
        r = requests.post(host + service,headers=authid,json=request)
        if (r.ok == False):
            print("SQL Execution error")
            print(r.json())
            return None
        
    except Exception as err:
        print("Error attempting RESTful call")
        print(err)
        return None
    
    jobid = r.json()['id']
    
    # Step 2: Wait for Answer set to complete
    
    service = "/sql_jobs/" + jobid
    rows = None
    while True:
        try:
            r = requests.get(host + service,headers=authid)
            if (r.ok == False):
                print("SQL answer set retrieval failed.")
                print(r.json())
                return None
            
        except Exception as err:
            print("Error attempting RESTful call")
            print(err)
            return None
        
        status = r.json()['status']
        if (status == "failed"):
            print("SQL failed")
            print(r.json())
            return None
                
        results = r.json()['results']
        try:
            columns = results[0]['columns']
            if (rows == None):
                rows = results[0]['rows']
            else:
                rows = rows + results[0]['rows']
        except:
            None
            
        if (status == 'completed'): break
            
    # Step 3: Create the Dataframe to return to the application
                    
    df = pandas.DataFrame(data=rows,columns=columns)
    cols = df.columns
    df[cols] = df[cols].apply(pandas.to_numeric, errors='ignore')
    
    return df

Now we can execute the previous SQL example with just a simple call.

In [ ]:
sql = 'select tabname, card, npages, fpages from syscat.tables fetch first 10 rows only'
restSQL(db2id,sql)

### Now it's your turn
Here you can now try out what you learned above and run your own SQL queries

In [ ]:
sql = 'select ...'
restSQL(db2id,sql)

## Summary
Using RESTful calls to Db2 removes much of the complexity of communicating with the database. There are no drivers required, no configuration file, nor any administration required on the client that is communicating with the database. All communication is done using RESTful API calls, which are available on all browsers and all operating systems. The calls to the database are replaced with standard POST and GET requests. Enabling RESTful support to Db2 opens up the type of applications that you can write and clients that you can connect to Db2 with.

#### Credits: 
IBM 2020, George Baklarz [baklarz@ca.ibm.com], Andreas Weininger [andreas.weininger@de.ibm.com], Stefan Hummel [stefan.hummel@de.ibm.com]